In [13]:
import numpy as np
import cv2
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib tk
import matplotlib as mpl

### Read output angle data (test folder 24, 30, 31, 32, 33, 35 from Preliminary result v5)

### Reducing the dimension of images

In [4]:
# read video frame
def dep_cam_reduced(dep_v_name):
    dep_v = cv2.VideoCapture(dep_v_name)
    ret, frame = dep_v.read()
    counter=0

    frame_count = int(dep_v.get(cv2.CAP_PROP_FRAME_COUNT))

#     frame_height = int(frame.shape[0]/5)
#     frame_width = int(frame.shape[1]/5)
#     frame_height = frame.shape[0]
#     frame_width = frame.shape[1]
    frame_height = 30
    frame_width = 40
    depth_frames = np.empty((frame_count, frame_height, frame_width))

    while(dep_v.isOpened()):
        ret, frame = dep_v.read()
        if ret == False:
            break
        if ret == True:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.resize(gray_frame, (frame_width, frame_height), interpolation = cv2.INTER_AREA)
            depth_frames[counter] = gray_frame
            counter+=1

    dep_v.release()
    cv2.destroyAllWindows()
    return depth_frames

In [5]:
# read output depth images data
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31', 'test32','test33', 'test35' ]
test_num = ['24', '30', '31', '32', '33', '35']
sub_name = foldername + test_folder[0]
dep_name = sub_name + r'/depth_processed_leo_test' + test_num[0] + '.avi';
depth =  dep_cam_reduced(dep_name)
for i in range(1, 6):
    subfolder_name = foldername + test_folder[i]
    dep_video_name = subfolder_name + r'/depth_processed_leo_test' + test_num[i] + '.avi'
    #print(dep_video_name)
    dep_v_temp =  dep_cam_reduced(dep_video_name)/255.0
    depth = np.concatenate((depth, dep_v_temp))

In [ ]:
print(depth.shape)
print(theta_z.shape)

In [ ]:
# Read force data
subfolder_name = foldername + test_folder[0]
force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[0] + '_' + '11_15_2020' + '.txt'
force = pd.read_csv(force_file)
force = force.iloc[:,:].values
for i in range(1,6):
    subfolder_name = foldername + test_folder[i]
    if test_folder[i] == 'test30' or test_folder[i] == 'test31':
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_24_2020' + '.txt'
    else: 
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_25_2020' + '.txt'
    dataset_y =  pd.read_csv(force_file)
    force_temp = dataset_y.iloc[:,:].values
    force = np.concatenate((force, force_temp))

### Comments:

###### Observation: Comparing to the NN that uses tanh and sigmoid activation function, the rectified linear unit fucntion seemed to have better performance (lower mae) when the layers number are the same. Also, the choice of metrics of model evaluation seemed to be important (mse is quite large compared to mae). But I wonder what happened to my plots ;(     

###### Also, right now I am using a dimension of 120 * 160, performing a dimensionality reduction could be helpful for extracting necessary features. A longer epoch time may also improve performance. Tried sklearn neural network MLPRegressor but does not work

In [24]:
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth, force, theta_z, test_size = 0.2, random_state = 0)

In [42]:
# dropout at both visible layer and hidden layer

import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model_dp = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 40)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model_dp.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model_dp.fit(depth_train, theta_z_train, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (depth_test, theta_z_test))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 9.6339 - mae: 9.6339 - val_loss: 9.3093 - val_mae: 9.3093
Epoch 2/300
515/515 [==============================] - 1s 2ms/step - loss: 9.2564 - mae: 9.2564 - val_loss: 9.1086 - val_mae: 9.1086
Epoch 3/300
515/515 [==============================] - 1s 1ms/step - loss: 9.0410 - mae: 9.0410 - val_loss: 8.8175 - val_mae: 8.8175
Epoch 4/300
515/515 [==============================] - 1s 1ms/step - loss: 8.9281 - mae: 8.9281 - val_loss: 8.8209 - val_mae: 8.8209
Epoch 5/300
515/515 [==============================] - 1s 1ms/step - loss: 8.7499 - mae: 8.7499 - val_loss: 8.5831 - val_mae: 8.5831
Epoch 6/300
515/515 [==============================] - 1s 1ms/step - loss: 8.5872 - mae: 8.5872 - val_loss: 8.3356 - val_mae: 8.3356
Epoch 7/300
515/515 [==============================] - 1s 1ms/step - loss: 8.4767 - mae: 8.4767 - val_loss: 8.2559 - val_mae: 8.2559
Epoch 8/300
515/515 [==============================] - 1s 1ms/step - 

In [26]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']
mpl.rcParams['figure.figsize'] = (20,5)
# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [43]:
y_prediction = model_dp.predict(depth_test)


mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(theta_z_test)
plt.plot(y_prediction,'--')
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample')
plt.ylabel('Angle (degree)')


Text(0, 0.5, 'Angle (degree)')

In [44]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(theta_z_test, y_prediction)

6.741460756272304

### Mulitple inputs in neural network

In [18]:
from sklearn.model_selection import train_test_split
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth, force, theta_z, test_size = 0.2, random_state = 0)

In [48]:
from tensorflow.keras import layers
input_depth = layers.Input(shape = (30, 40))
input_force = layers.Input(shape = (6, ))

# first branch
x = layers.Flatten(input_shape=(30, 40))(input_depth)
x = layers.Dense(units=128, activation='relu')(x)
x = layers.Dense(units=64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = tf.keras.Model(inputs= input_depth, outputs = x)

# second branch
y = layers.Dense(units=128, activation='relu')(input_force)
y = layers.Dense(units=64, activation='relu')(y)
y = layers.Dense(32, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = tf.keras.Model(inputs= input_force, outputs = y)

# combine two branches
combined = layers.concatenate([x.output, y.output])

# regression to one output
z = layers.Dense(units=32, activation='relu')(combined)
z = layers.Dropout(0.2)(z)
z = layers.Dense(units=16, activation='relu')(combined)
z = layers.Dense(1, activation='linear')(z)
model_fd = tf.keras.Model(inputs = [x.input, y.input], outputs = z)

#tf.keras.utils.plot_model(model, to_file = 'model.png')

In [49]:
model_fd.compile(optimizer='adam', loss= 'mae', metrics=['mae'])

history = model_fd.fit([depth_train, force_train],theta_z_train, verbose = 1, epochs = 300, batch_size = 128,
                   validation_data = ([depth_test, force_test], theta_z_test))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 7.9915 - mae: 7.9915 - val_loss: 7.4197 - val_mae: 7.4197
Epoch 2/300
515/515 [==============================] - 1s 2ms/step - loss: 7.0734 - mae: 7.0734 - val_loss: 6.7159 - val_mae: 6.7159
Epoch 3/300
515/515 [==============================] - 1s 2ms/step - loss: 6.6195 - mae: 6.6195 - val_loss: 6.2822 - val_mae: 6.2822
Epoch 4/300
515/515 [==============================] - 1s 2ms/step - loss: 6.2594 - mae: 6.2594 - val_loss: 5.9572 - val_mae: 5.9572
Epoch 5/300
515/515 [==============================] - 1s 2ms/step - loss: 5.9354 - mae: 5.9354 - val_loss: 5.8057 - val_mae: 5.8057
Epoch 6/300
515/515 [==============================] - 1s 2ms/step - loss: 5.7589 - mae: 5.7589 - val_loss: 5.5760 - val_mae: 5.5760
Epoch 7/300
515/515 [==============================] - 1s 2ms/step - loss: 5.8777 - mae: 5.8777 - val_loss: 5.5381 - val_mae: 5.5381
Epoch 8/300
515/515 [==============================] - 1s 2ms/step - 

In [53]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [88]:
y_prediction = model_fd.predict([depth_test, force_test])

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,7)
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18)
plt.plot(theta_z_test)
plt.plot(y_prediction,'--')
plt.xlim([4900, 5200])
plt.ylim([-110, 110])
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample', fontsize=18)
plt.ylabel('Angle (degree)', fontsize=18)
plt.show()
plt.savefig('ground_v_prediction_zoomed in.png', dpi = 300)

In [81]:
mpl.rcParams['figure.figsize'] = (15,10)
ax1 = plt.subplot(211)
ax1.plot(theta_z_test)
ax1.plot(y_prediction,'--')
plt.legend(['Ground Truth', 'Prediction'])
plt.ylabel('Angle (degree)')

ax2 = plt.subplot(212)
ax2.plot(theta_z_test)
ax2.plot(y_prediction,'--')
ax2.set_xlim([4900, 5200])
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample')
plt.ylabel('Angle (degree)')
plt.show()
plt.savefig('ground_v_prediction.png', dpi = 300)

In [52]:
mean_absolute_error(theta_z_test, y_prediction)

3.104436460064926

In [ ]:
plt.scatter(y_prediction, theta_z_test)
plt.xlabel('Prediction angle')
plt.ylabel('Ground angle')
plt.show()

### Feed only force data to the NN

In [39]:
import tensorflow as tf
from keras.utils.vis_utils import plot_model
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(force_train, theta_z_train, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (force_test, theta_z_test))

Epoch 1/300
515/515 [==============================] - 0s 885us/step - loss: 8.8208 - mae: 8.8208 - val_loss: 7.7199 - val_mae: 7.7199
Epoch 2/300
515/515 [==============================] - 0s 776us/step - loss: 7.7106 - mae: 7.7106 - val_loss: 7.4109 - val_mae: 7.4109
Epoch 3/300
515/515 [==============================] - 0s 756us/step - loss: 7.4826 - mae: 7.4826 - val_loss: 7.3380 - val_mae: 7.3380
Epoch 4/300
515/515 [==============================] - 0s 761us/step - loss: 7.4321 - mae: 7.4321 - val_loss: 7.3043 - val_mae: 7.3043
Epoch 5/300
515/515 [==============================] - 0s 777us/step - loss: 7.3942 - mae: 7.3942 - val_loss: 7.3428 - val_mae: 7.3428
Epoch 6/300
515/515 [==============================] - 0s 759us/step - loss: 7.3432 - mae: 7.3432 - val_loss: 7.3509 - val_mae: 7.3509
Epoch 7/300
515/515 [==============================] - 0s 776us/step - loss: 7.3434 - mae: 7.3434 - val_loss: 7.2937 - val_mae: 7.2937
Epoch 8/300
515/515 [==============================] - 

In [30]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [45]:
y_prediction = model.predict(force_test)


mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(theta_z_test)
plt.plot(y_prediction,'--')
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample')
plt.ylabel('Angle (degree)')

Text(0, 0.5, 'Angle (degree)')

In [46]:
mean_absolute_error(theta_z_test, y_prediction)

6.913944966483415